In [1]:
#function calling을 사용하지 않으면 USA와 Sourth Korea에 대한 답변 format이 달라진다.
#첫 번째 응답은 주를 숫자(한 국가의 총 주 수)로 반환하고, 두 번째 응답은 주를 배열(주)로 반환

In [20]:
import os
import openai
import json
from openai import OpenAI

client = OpenAI(
  api_key="sk-"
)

prompt = '''
인구, 수도, 주, 미국. JSON 객체로 반환
'''

completion = client.chat.completions.create(
    model = "gpt-3.5-turbo-0613",
    messages = [{'role': 'user', 'content': prompt}],
    max_tokens = 1024,
    temperature = 0
)

print(completion)
json.loads(completion.choices[0].message.content)

ChatCompletion(id='chatcmpl-9D3Ux6Ftybn4EAMB7pHWE8RObRhvV', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "인구": "331,002,651",\n  "수도": "워싱턴 D.C.",\n  "주": "50",\n  "미국": true\n}', role='assistant', function_call=None, tool_calls=None))], created=1712898739, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=43, prompt_tokens=27, total_tokens=70))


{'인구': '331,002,651', '수도': '워싱턴 D.C.', '주': '50', '미국': True}

In [21]:
prompt = '''
인구, 수도, 주, 대한민국. JSON 객체로 반환
'''

In [22]:
completion = client.chat.completions.create(
    model = "gpt-3.5-turbo-0613",
    messages = [{'role': 'user', 'content': prompt}],
    max_tokens = 1024,
    temperature = 0
)

print(completion)
json.loads(completion.choices[0].message.content)

ChatCompletion(id='chatcmpl-9D3UzPxCOpyMQsL6AHxW27RSqf2AJ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "인구": "51,709,098명",\n  "수도": "서울",\n  "주": "대한민국",\n  "대한민국": {\n    "인구": "51,709,098명",\n    "수도": "서울",\n    "주": "대한민국"\n  }\n}', role='assistant', function_call=None, tool_calls=None))], created=1712898741, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=84, prompt_tokens=28, total_tokens=112))


{'인구': '51,709,098명',
 '수도': '서울',
 '주': '대한민국',
 '대한민국': {'인구': '51,709,098명', '수도': '서울', '주': '대한민국'}}

In [23]:
#첫 번째 쿼리에서는 인구가 정수(331002651)로 반환되고, 두 번째 쿼리에서는 인구가 문자열('32,730,000')로 반환
#첫 번째 응답은 주를 숫자(한 국가의 총 주 수)로 반환하고, 두 번째 응답은 주를 배열(주)로 반환
#즉, 응답의 불일치가 발생 

In [24]:
test_functions = [
    {
        'name': 'get_country_details',
        'description': 'Get the details of a country',
        'parameters': {
            'type': 'object',
            'properties': {                
                'Population': {
                    'type': 'integer',
                    'description': '인구'
                },                
                'Capital': {
                    'type': 'string',
                    'description': '수도'
                },
                'States': {
                    'type': 'string',
                    'description': '주'
                },
            }
        }
    }
]

In [25]:
import openai

country = '미국'
prompt = f'''
Population, Capital, States of {country}.
'''

completion = client.chat.completions.create(
    model = 'gpt-4',
    messages = [{"role": "user", "content": prompt}],
    functions = test_functions,   # <========add this parameter
    function_call = 'auto',     # <========add this parameter
    max_tokens = 1024,
    temperature = 0
)

print(completion)

ChatCompletion(id='chatcmpl-9D3VYYoOYmLVZtxX3wajivlpPoy5x', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "Population": 331002651,\n  "Capital": "Washington, D.C.",\n  "States": "50"\n}', name='get_country_details'), tool_calls=None))], created=1712898776, model='gpt-4-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=35, prompt_tokens=79, total_tokens=114))


In [26]:
import json
print(json.loads(completion.choices[0].message.function_call.arguments))

{'Population': 331002651, 'Capital': 'Washington, D.C.', 'States': '50'}


In [27]:
import openai

country = '대한민국'
prompt = f'''
Population, Capital, States of {country}.
'''

completion = client.chat.completions.create(
    model = 'gpt-4',
    messages = [{"role": "user", "content": prompt}],
    functions = test_functions,   # <========add this parameter
    function_call = 'auto',     # <========add this parameter
    max_tokens = 1024,
    temperature = 0
)

print(completion)

ChatCompletion(id='chatcmpl-9D3VaMRZzWzlpvS3kC1ZMAaUcK8M4', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "Population": 51640000,\n  "Capital": "서울",\n  "States": "9도 8특별시 1특별자치시 1특별자치도"\n}', name='get_country_details'), tool_calls=None))], created=1712898778, model='gpt-4-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=60, prompt_tokens=80, total_tokens=140))


In [28]:
import json
print(json.loads(completion.choices[0].message.function_call.arguments))

{'Population': 51640000, 'Capital': '서울', 'States': '9도 8특별시 1특별자치시 1특별자치도'}
